In [1]:
import os 
import numpy as np
import pandas as pd
import glob
import json
import tqdm
import warnings
from multiprocessing import Pool
warnings.filterwarnings(action= 'ignore')

In [2]:
data_path = '../dataset/'
data_all_path = []
for i in ['train' , 'val']:
    for j in [ 'label']: # add img is available
        for k in ['large' , 'medium' , 'small']:
            for p in ['illegal' ,'normal' ]:
                data_full_path = os.path.join(data_path , i , j , k , p)
                data_all_path.extend(glob.glob(data_full_path  + '/*'))

print('total number of data : ' , len(data_all_path))

total number of data :  369236


In [42]:
# 데이터를 병렬으로 불러오기 위한 function
def parallelize_dataframe(df, func , num_cores):

    # 데이터 나누고, 쓰레드 할당받기
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    result = pool.map(func, df_split)
    pool.close()
    pool.join()

    # 결과 합치기
    result_1 , result_2 = [] , []
    for i in result:
        result_1.append(i[0])
        result_2.append(i[1])
    df_result = pd.concat(result_1 , axis = 0 )
    df_item = pd.concat(result_2 , axis = 0 )

    return df_result , df_item

# json to df
def json_loader(data_path):
    df = pd.DataFrame()
    df_item = pd.DataFrame()
    df_error = []

    for i ,data in enumerate(data_path):
        with open(data, 'r') as json_read :
            json_str = json.load(json_read) # json.load 로 파일 읽기

            # get img info
            df = df.append(json_str['FILE'][0] , ignore_index= True)
            for json_item in json_str['FILE'][0]['ITEMS']:
                json_item['FILE_NAME'] = json_str['FILE'][0]['FILE_NAME']

                # get obj info
                df_item = df_item.append(json_item , ignore_index= True)
                
    return df , df_item

In [55]:
df , df_item = parallelize_dataframe(data_all_path , json_loader , 30)

In [66]:
df.to_csv('img_info.csv' , index = False)
df_item.to_csv('obj_info.csv' , index = False)